# Units

[astropy.units]: https://docs.astropy.org/en/stable/units/index.html
[plasmapy.particles]: ../particles/index.rst
[plasmapy.formulary]: ../formulary/index.rst

This notebook introduces [astropy.units] with an emphasis on the functionality needed to work with [plasmapy.particles] and [plasmapy.formulary].

## Getting started with Astropy units

In scientific computing, we often represent physical quantities as numbers.

In [ ]:
distance_in_miles = 50
time_in_hours = 2
velocity_in_mph = distance_in_miles / time_in_hours
print(velocity_in_mph)

Representing a physical quantity as a number has risks. We might accidentally perform operations with different units, like `time_in_seconds + time_in_hours`. We might even accidentally perform operations with physically incompatible units, like `length + time` without catching our mistake.

[astropy.units]: https://docs.astropy.org/en/stable/units/index.html

Using [astropy.units] helps us avoid these problems. We typically import this subpackage as `u`.

In [ ]:
import astropy.units as u

We can create a physical quantity by multiplying a number or array with a unit.

In [ ]:
distance = 80 * u.km
print(distance)

[Quantity]: https://docs.astropy.org/en/stable/api/astropy.units.Quantity.html#astropy.units.Quantity

This operation creates a [Quantity]: a number, sequence, or array that has been assigned a physical unit.

In [ ]:
type(distance)

[Quantity]: https://docs.astropy.org/en/stable/api/astropy.units.Quantity.html#astropy.units.Quantity

We can also create an object by using the [Quantity] class itself.

In [ ]:
time = u.Quantity(120, u.min)

[Quantity]: https://docs.astropy.org/en/stable/api/astropy.units.Quantity.html#astropy.units.Quantity

We can create [Quantity] objects with compound units.

In [ ]:
88 * u.imperial.mile / u.hour

[.value]: https://docs.astropy.org/en/stable/api/astropy.units.Quantity.html#astropy.units.Quantity.value 
[Quantity]: https://docs.astropy.org/en/stable/api/astropy.units.Quantity.html#astropy.units.Quantity

The [.value] attribute of a [Quantity] shows the number or array without the unit.

In [ ]:
time.value

[.unit]: https://docs.astropy.org/en/stable/api/astropy.units.Quantity.html#astropy.units.Quantity.unit
[Quantity]: https://docs.astropy.org/en/stable/api/astropy.units.Quantity.html#astropy.units.Quantity

The [.unit] attribute of a [Quantity] shows the unit.

In [ ]:
time.unit

[Quantity]: https://docs.astropy.org/en/stable/api/astropy.units.Quantity.html#astropy.units.Quantity

Operations between [Quantity] objects handle unit conversions automatically. We can add [Quantity] objects together as long as their units have the same physical type.

In [ ]:
1 * u.m + 2 * u.cm

Units get handled automatically during operations like multiplication, division, and exponentiation.

In [ ]:
velocity = distance / time
print(velocity)

[Quantity]: https://docs.astropy.org/en/stable/api/astropy.units.Quantity.html#astropy.units.Quantity

We can even create [Quantity] objects that are dimensionless.

In [ ]:
3 * u.dimensionless_unscaled

[Quantity]: https://docs.astropy.org/en/stable/api/astropy.units.Quantity.html#astropy.units.Quantity
[.to]: https://docs.astropy.org/en/stable/api/astropy.units.Quantity.html#astropy.units.Quantity.to

The [.to] method allows us to convert a [Quantity] to different units of the same physical type. This method accepts strings that represent a unit (including compound units) or a unit object.

In [ ]:
velocity.to("m/s")

In [ ]:
velocity.to(u.m / u.s)

[Quantity]: https://docs.astropy.org/en/stable/api/astropy.units.Quantity.html#astropy.units.Quantity
[.si]: https://docs.astropy.org/en/stable/api/astropy.units.Quantity.html#astropy.units.Quantity.si
[.cgs]: https://docs.astropy.org/en/stable/api/astropy.units.Quantity.html#astropy.units.Quantity.cgs

The [.si] and [.cgs] attributes convert the [Quantity] to SI or CGS units, respectively. 

In [ ]:
velocity.si

In [ ]:
velocity.cgs

Attempting an operation between physically incompatible units gives us an error, which we can use to find bugs in our code.

In [ ]:
3 * u.m + 3 * u.s

[performance tips]: https://docs.astropy.org/en/stable/units/index.html#performance-tips
[astropy.units]: https://docs.astropy.org/en/stable/units/index.html

Astropy's documentation includes [performance tips] for using [astropy.units] in computationally intensive applications. For example, in computationally intensive operations, it helps to put compound units in parentheses.

In [ ]:
volume = 0.62 * (u.barn * u.Mpc)

## Using Astropy units in PlasmaPy

[astropy.units]: https://docs.astropy.org/en/stable/units/index.html
[plasmapy.particles]: ../particles/index.rst
[Quantity]: https://docs.astropy.org/en/stable/api/astropy.units.Quantity.html#astropy.units.Quantity
[Particle]: ../api/plasmapy.particles.particle_class.Particle.rst

Now we can show some uses of [astropy.units] in PlasmaPy.  We can start with [plasmapy.particles]. Many of the attributes of the [Particle] class provide [Quantity] objects.

In [ ]:
from plasmapy.particles import Particle

alpha = Particle("He-4 2+")

In [ ]:
alpha.charge

In [ ]:
alpha.mass

[Quantity]: https://docs.astropy.org/en/stable/api/astropy.units.Quantity.html#astropy.units.Quantity
[plasmapy.formulary]: ../formulary/index.rst

Similarly, [Quantity] objects are the expected inputs and outputs of most functions in [plasmapy.formulary]. We can use them to calculate some plasma parameters for a typical region of the solar corona.

In [ ]:
from plasmapy.formulary import Alfven_speed, gyrofrequency

In [ ]:
B = 0.01 * u.T
n = 1e15 * u.m ** -3
proton = Particle("p+")

In [ ]:
Alfven_speed(B=B, density=n, ion=proton).to("km /s")

In [ ]:
gyrofrequency(B=B, particle="e-")

## Equivalencies

[electron-volt]: https://en.wikipedia.org/wiki/Electronvolt
[Boltzmann constant]: https://en.wikipedia.org/wiki/Boltzmann_constant

Plasma scientists often the [electron-volt] (eV) as a unit of temperature. This is a shortcut for describing the thermal energy per particle, or more accurately the temperature multiplied by the [Boltzmann constant], $k_B$. Because an electron-volt is a unit of energy rather than temperature, we cannnot directly convert electron-volts to kelvin.

In [ ]:
u.eV.to("K")

[astropy.units]: https://docs.astropy.org/en/stable/units/index.html
[equivalencies]: https://docs.astropy.org/en/stable/units/equivalencies.html
[temperature_energy()]: https://docs.astropy.org/en/stable/units/equivalencies.html#temperature-energy-equivalency

To handle situations like this, [astropy.units] has built-in [equivalencies]. The conversion from eV to K can be done by using the [temperature_energy()] equivalency.

In [ ]:
(1 * u.eV).to("K", equivalencies=u.temperature_energy())

[dimensionless_angles()]: https://docs.astropy.org/en/stable/api/astropy.units.equivalencies.dimensionless_angles.html#dimensionless-angles

Radians are treated dimensionlessly by using the [dimensionless_angles()] equivalency. Note that this equivalency does not account for the multiplicative factor of $2π$ that is used when converting between frequency and angular frequency.

In [ ]:
(3.2 * u.rad / u.s).to("1 / s", equivalencies=u.dimensionless_angles())

## Using Astropy constants

[astropy.constants]: https://docs.astropy.org/en/stable/constants/index.html

We can use [astropy.constants] to access the most commonly needed physical constants.

In [ ]:
from astropy.constants import c, e, k_B

print(c)

[Constant]: https://docs.astropy.org/en/stable/api/astropy.constants.Constant.html#astropy.constants.Constant
[Quantity]: https://docs.astropy.org/en/stable/api/astropy.units.Quantity.html#astropy.units.Quantity
[u.temperature_energy()]: https://docs.astropy.org/en/stable/units/equivalencies.html#temperature-energy-equivalency

A [Constant] behaves very similarly to a [Quantity]. For example, we can use the Boltzmann constant to mimic the behavior of [u.temperature_energy()].

In [ ]:
thermal_energy_per_particle = 0.6 * u.keV
temperature = thermal_energy_per_particle / k_B
print(temperature.to("MK"))

Electromagnetic constants often need the unit system to be specified. Code within PlasmaPy generally uses SI units.

In [ ]:
100 * e

In [ ]:
100 * e.si

## Physical types

[physical type]: https://docs.astropy.org/en/stable/units/physical_types.html
[physical_type]: https://docs.astropy.org/en/stable/api/astropy.units.UnitBase.html#astropy.units.UnitBase.physical_type
[get_physical_type()]: https://docs.astropy.org/en/stable/api/astropy.units.get_physical_type.html#astropy.units.get_physical_type

A [physical type] corresponds to physical quantities with dimensionally compatible units. Astropy has functionality that represents different physical types. These physical type objects can be accessed using either the [physical_type] attribute of a unit or [get_physical_type()].

In [ ]:
(u.m ** 2 / u.s).physical_type

In [ ]:
u.get_physical_type("number density")

These physical type objects can be used for dimensional analysis.

In [ ]:
energy_density = (u.J * u.m ** -3).physical_type
velocity = u.get_physical_type("velocity")
energy_density * velocity